In [1]:
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder

In [2]:
# Read the diamonds_train.db file
conn = sqlite3.connect('diamonds_train.db')

# Fetch the required tables
df_dim = pd.read_sql_query('SELECT * FROM diamonds_dimensions', conn)
df_trans = pd.read_sql_query('SELECT * FROM diamonds_transactional', conn)
df_prop = pd.read_sql_query('SELECT * FROM diamonds_properties', conn)
df_cut = pd.read_sql_query('SELECT * FROM diamonds_cut', conn)
df_color = pd.read_sql_query('SELECT * FROM diamonds_color', conn)
df_clarity = pd.read_sql_query('SELECT * FROM diamonds_clarity', conn)
df_city = pd.read_sql_query('SELECT * FROM diamonds_city', conn)

# Merge the tables based on index_id
df = pd.merge(df_dim, df_trans, on='index_id')
df = pd.merge(df, df_prop, on='index_id')
df = pd.merge(df, df_cut, on='cut_id')
df = pd.merge(df, df_color, on='color_id')
df = pd.merge(df, df_clarity, on='clarity_id')
df = pd.merge(df, df_city, on='city_id')

# Create X and y
X = df[['carat', 'cut', 'color', 'clarity', 'x', 'y', 'z', 'depth', 'table', 'city']]
y = df['price']

# Close the database connection
conn.close()

print("DataFrame:")
print(df.head())

print("\nX:")
print(X.head())

print("\ny:")
print(y.head())

DataFrame:
                                            index_id  depth  table     x  \
0  5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...   62.4   58.0  6.83   
1  41667f6e2629360aecaf00b20f8732e3310417ebd54b24...   61.6   58.0  6.40   
2  01f8667f50d52677bea23231a74156e4f92360d7bc3db6...   62.3   58.0  5.86   
3  c3867352aab641358faec75d733af012dbe2259a014ea8...   59.6   60.0  7.58   
4  0da4b104c4d8589fcb96a03aa0787549a2631935b0f499...   60.2   62.0  5.40   

      y     z  price                                            city_id  \
0  6.79  4.25   4268  6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...   
1  6.35  3.93   3513  6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...   
2  5.80  3.63   1792  6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...   
3  7.48  4.49   7553  6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...   
4  5.33  3.23   1176  6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...   

   carat                                             cut_id  \
0   1.21  de88c121

In [3]:
# Check for null values in the dataset
print(df.isnull().sum())

# Drop rows with null values
df.dropna(inplace=True)

index_id      0
depth         0
table         0
x             0
y             0
z             0
price         0
city_id       0
carat         0
cut_id        0
color_id      0
clarity_id    0
cut           0
color         0
clarity       0
city          0
dtype: int64


In [4]:
from sklearn.preprocessing import OrdinalEncoder

# Initialize the encoders
encoder_cut = OrdinalEncoder()
encoder_color = OrdinalEncoder()
encoder_clarity = OrdinalEncoder()
encoder_city = OrdinalEncoder()

# Encode categorical variables
X = X.copy()
X.loc[:, 'cut'] = encoder_cut.fit_transform(X[['cut']])
X.loc[:, 'color'] = encoder_color.fit_transform(X[['color']])
X.loc[:, 'clarity'] = encoder_clarity.fit_transform(X[['clarity']])
X.loc[:, 'city'] = encoder_city.fit_transform(X[['city']])

C:\Users\bfven\AppData\Local\Temp\ipykernel_36092\1869066263.py:11: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.loc[:, 'cut'] = encoder_cut.fit_transform(X[['cut']])
C:\Users\bfven\AppData\Local\Temp\ipykernel_36092\1869066263.py:12: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.loc[:, 'color'] = encoder_color.fit_transform(X[['color']])
C:\Users\bfven\AppData\Local\Temp\ipykernel_36092\1869066263.py:13: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting 

In [5]:
# Scale numerical features
scaler = StandardScaler()
X[['carat', 'x', 'y', 'z', 'depth', 'table']] = scaler.fit_transform(X[['carat', 'x', 'y', 'z', 'depth', 'table']])

In [6]:
X

,carat,cut,color,clarity,x,y,z,depth,table,city
0,0.867006,3.0,6.0,5.0,0.978807,0.921985,1.022657,0.452019,0.247981,2.0
1,0.467458,3.0,6.0,5.0,0.596394,0.538254,0.563582,-0.106755,0.247981,2.0
2,-0.058262,3.0,6.0,5.0,0.116154,0.058590,0.133198,0.382172,0.247981,2.0
3,1.497870,3.0,6.0,5.0,1.645806,1.523746,1.366964,-1.503688,1.143433,2.0
4,-0.478838,3.0,6.0,5.0,-0.292939,-0.351305,-0.440646,-1.084608,2.038886,2.0
...,...,...,...,...,...,...,...,...,...,...
40450,-0.541924,2.0,2.0,1.0,-0.435232,-0.403632,-0.383262,0.312325,-1.542924,10.0
40451,-0.562953,2.0,2.0,1.0,-0.453019,-0.421074,-0.426300,0.102785,-1.542924,10.0
40452,-1.046615,2.0,2.0,1.0,-1.271205,-1.214700,-1.215336,0.382172,-1.095198,10.0
40453,-1.130730,2.0,2.0,1.0,-1.404604,-1.310633,-1.416182,-0.595681,-1.095198,10.0


In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

LinearRegression()

In [8]:
from sklearn.ensemble import RandomForestRegressor

# Create and train the model
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [9]:
y_predict = model.predict(X_test)
y_predict

array([8239.55,  502.5 , 8354.45, ..., 9372.36, 3746.49, 4692.26])

In [10]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_predict, squared=False)
rmse

601.2113184511185

In [11]:
diamantes_test = pd.read_csv('diamonds_test.csv')
diamantes_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [12]:
print("\nX:")
print(X.head())


X:
      carat  cut  color  clarity         x         y         z     depth  \
0  0.867006  3.0    6.0      5.0  0.978807  0.921985  1.022657  0.452019   
1  0.467458  3.0    6.0      5.0  0.596394  0.538254  0.563582 -0.106755   
2 -0.058262  3.0    6.0      5.0  0.116154  0.058590  0.133198  0.382172   
3  1.497870  3.0    6.0      5.0  1.645806  1.523746  1.366964 -1.503688   
4 -0.478838  3.0    6.0      5.0 -0.292939 -0.351305 -0.440646 -1.084608   

      table  city  
0  0.247981   2.0  
1  0.247981   2.0  
2  0.247981   2.0  
3  1.143433   2.0  
4  2.038886   2.0  


In [ ]:
# Check for null values in the dataset
print(diamantes_test.isnull().sum())

# Drop rows with null values
diamantes_test.dropna(inplace=True)

# Remove commas and convert numeric columns to float
numeric_cols = ['carat', 'x', 'y', 'z', 'depth', 'table']
for col in numeric_cols:
    diamantes_test[col] = diamantes_test[col].astype(str).str.replace(',', '').astype(float)

# Reorder the columns to match the order used during training
columns_order = ['carat', 'cut', 'color', 'clarity', 'x', 'y', 'z', 'depth', 'table', 'city']
diamantes_test = diamantes_test[columns_order]

In [ ]:
# Encode categorical variables

diamantes_test.loc[:, 'cut'] = encoder_cut.fit_transform(diamantes_test[['cut']])
diamantes_test.loc[:, 'color'] = encoder_color.fit_transform(diamantes_test[['color']])
diamantes_test.loc[:, 'clarity'] = encoder_clarity.fit_transform(diamantes_test[['clarity']])
diamantes_test.loc[:, 'city'] = encoder_city.fit_transform(diamantes_test[['city']])

In [ ]:
diamantes_test.head()

In [ ]:
X_test.columns

In [ ]:
diamantes_test.columns

In [ ]:
diamantes_test = diamantes_test[['carat', 'cut', 'color', 'clarity', 'x', 'y', 'z', 'depth', 'table',
       'city']]

In [ ]:
# Scale numerical features
scaler = StandardScaler()
diamantes_test[['carat', 'x', 'y', 'z', 'depth', 'table']] = scaler.fit_transform(diamantes_test[['carat', 'x', 'y', 'z', 'depth', 'table']])

In [ ]:
y_final = model.predict(diamantes_test)
y_final

In [ ]:
submission = pd.DataFrame(y_final)
submission

In [ ]:
# Rename the columns
submission.rename(columns={'A': 'id', 'B': 'price'}, inplace=True)

# Display the updated DataFrame
print(submission.head())

In [ ]:
submission.head

In [ ]:
# Rename the columns
submission = submission.rename(columns={0: 'id', 1: 'price'})

# Display the updated DataFrame
print(submission.head())


In [ ]:
submission.to_csv("./submission3.csv")